# Intent-Based Classification for RAG

The difference between demo RAG and production RAG is often one thing:
**Stop treating every query the same.**

This notebook walks through:
1. The problem with naive RAG
2. How intent classification works
3. Routing queries to different retrieval strategies
4. A complete working example

## The Problem

Naive RAG treats every query the same:

```
Query → Embed → Vector Search → Stuff into Prompt → Generate
```

But different queries need different approaches:

| Query | What it needs | Naive RAG does |
|-------|--------------|----------------|
| "What was Q3 revenue?" | Database lookup | Searches 50k document chunks |
| "How do I reset my API key?" | Exact steps, keywords matter | Returns vague conceptual text |
| "What is OAuth?" | Broad explanation | Works okay, but suboptimal |
| "Postgres vs MongoDB?" | Info on BOTH, then compare | Only retrieves one side |

## The Solution: Classify Before You Retrieve

Add ONE step before your RAG pipeline:

```
Query → CLASSIFY INTENT → Route to Strategy → Retrieve → Generate
```

The classification step is cheap (one fast LLM call, ~50ms) and determines which retrieval strategy to use.

In [ ]:
# First, let's look at the intent categories we'll use

from enum import Enum

class Intent(str, Enum):
    """The different types of queries a user might ask."""
    
    CONCEPTUAL = "conceptual"      # "What is X?" - needs broad context
    PROCEDURAL = "procedural"      # "How do I X?" - needs specific steps  
    FACTUAL = "factual"            # "What was X?" - needs data lookup
    COMPARATIVE = "comparative"    # "X vs Y?" - needs multi-source synthesis
    OUT_OF_SCOPE = "out_of_scope"  # Off-topic - don't search at all

## The Classification Prompt

Intent classification is just a specialized prompt. We ask a fast LLM to categorize the query.

In [ ]:
CLASSIFICATION_PROMPT = """You are a query classifier for a software documentation system.

Classify the user's query into exactly ONE category:

CONCEPTUAL - Questions about what something is or how it works at a high level.
Examples: "What is a JWT?", "Explain OAuth"

PROCEDURAL - Questions about how to do something specific, step-by-step.
Examples: "How do I reset my API key?", "Show me how to deploy"

FACTUAL - Questions asking for specific data, numbers, or lookups.
Examples: "What was our Q3 revenue?", "What's the API rate limit?"

COMPARATIVE - Questions comparing two or more options.
Examples: "Should I use Postgres or MongoDB?", "REST vs GraphQL?"

OUT_OF_SCOPE - Questions unrelated to our software/documentation.
Examples: "What's the weather?", "Tell me a joke"

Respond with ONLY the category name in uppercase. Nothing else.

Query: {query}"""

print("Classification prompt template created.")
print(f"\nExample for 'How do I reset my password?':\n")
print(CLASSIFICATION_PROMPT.format(query="How do I reset my password?"))

## Live Classification

Let's classify some real queries using the OpenAI Responses API.

In [ ]:
from openai import OpenAI

client = OpenAI()

def classify_query(query: str) -> str:
    """Classify a query into one of our intent categories."""
    response = client.responses.create(
        model="gpt-4o-mini",  # Fast model for classification
        input=CLASSIFICATION_PROMPT.format(query=query),
    )
    return response.output_text.strip()

# Test with different query types
test_queries = [
    "What is a JWT?",
    "How do I reset my API key?",
    "What was our Q3 revenue?",
    "Should I use Postgres or MongoDB?",
    "What's the weather like today?"
]

print("Query Classification Results:\n")
for query in test_queries:
    intent = classify_query(query)
    print(f"  '{query}'")
    print(f"  → {intent}\n")

## Routing to Different Retrieval Strategies

Once we know the intent, we route to the appropriate retrieval strategy.

Each strategy is optimized for its query type.

In [ ]:
# Import our mock retrieval strategies
from retrieval import (
    semantic_search,
    hybrid_search, 
    structured_query,
    multi_source_retrieval,
    early_exit
)

def route_to_retrieval(intent: str, query: str):
    """Route to the appropriate retrieval strategy based on intent."""
    
    match intent:
        case "CONCEPTUAL":
            # Broad understanding needed
            # Use semantic/vector search with larger chunks
            return semantic_search(query)
            
        case "PROCEDURAL":
            # Specific steps needed, keywords matter
            # Use hybrid search (vector + keyword)
            return hybrid_search(query)
            
        case "FACTUAL":
            # Data lookup needed
            # Skip vectors entirely - query database directly
            return structured_query(query)
            
        case "COMPARATIVE":
            # Need info on multiple items
            # Retrieve from multiple sources, then synthesize
            return multi_source_retrieval(query)
            
        case "OUT_OF_SCOPE":
            # Don't search at all
            # Return canned response immediately
            return early_exit(query)
            
        case _:
            # Fallback to semantic search
            return semantic_search(query)

## Complete Example: Query → Classify → Route → Retrieve

Let's put it all together and trace through a few queries.

In [ ]:
def process_query(query: str) -> dict:
    """Complete pipeline: classify → route → retrieve."""
    
    # Step 1: Classify
    intent = classify_query(query)
    
    # Step 2: Route to retrieval
    result = route_to_retrieval(intent, query)
    
    return {
        "query": query,
        "intent": intent,
        "strategy": result.strategy_used,
        "chunks": result.chunks,
        "metadata": result.metadata
    }

# Process a FACTUAL query
result = process_query("What was our Q3 revenue?")

print(f"Query: {result['query']}")
print(f"Intent: {result['intent']}")
print(f"Strategy: {result['strategy']}")
print(f"Retrieved: {result['chunks']}")
print(f"\nNote: This skipped vector search entirely!")

In [ ]:
# Process a PROCEDURAL query
result = process_query("How do I reset my API key?")

print(f"Query: {result['query']}")
print(f"Intent: {result['intent']}")
print(f"Strategy: {result['strategy']}")
print(f"\nRetrieved steps:")
print(result['chunks'][0][:500] + "...")

In [ ]:
# Process a COMPARATIVE query
result = process_query("Should I use Postgres or MongoDB?")

print(f"Query: {result['query']}")
print(f"Intent: {result['intent']}")
print(f"Strategy: {result['strategy']}")
print(f"\nRetrieved comparison data:")
for chunk in result['chunks']:
    print(f"  {chunk}")

In [ ]:
# Process an OUT_OF_SCOPE query
result = process_query("What's the weather like today?")

print(f"Query: {result['query']}")
print(f"Intent: {result['intent']}")
print(f"Strategy: {result['strategy']}")
print(f"\nResponse: {result['chunks'][0]}")
print(f"\nMetadata: {result['metadata']}")
print("\nNote: No search performed! Early exit saves tokens and latency.")

## Why This Makes Your System FASTER

Counter-intuitive: adding a classification step often reduces latency.

**Without routing (naive RAG):**
Every query runs the full pipeline:
- Embed query (~50ms)
- Vector search (~100ms)
- Maybe rerank (~200ms)
- Generate answer (~500ms)
- **Total: ~850ms**

**With routing:**
- Classification (~50ms)
- Then ONE of:
  - Factual → SQL query (~20ms) → Skip expensive LLM
  - Out of scope → Cached response (~0ms)
  - Procedural → Targeted hybrid search (~80ms)
  - etc.

You're not adding latency. You're adding a cheap check that skips expensive operations.

## Summary

1. **The Problem**: Naive RAG uses the same retrieval for all query types

2. **The Solution**: Classify intent BEFORE you retrieve

3. **The Categories**:
   - CONCEPTUAL → Semantic search with large chunks
   - PROCEDURAL → Hybrid search (keywords matter)
   - FACTUAL → Skip vectors, query database directly
   - COMPARATIVE → Multi-source retrieval
   - OUT_OF_SCOPE → Early exit, don't search

4. **The Benefit**: Better accuracy AND often lower latency

This is one of the key patterns that separates demo RAG from production RAG.